In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
import os
import pandas as pd
from mlrun.datastore import DataItem
from mlrun.artifacts import get_model
from cloudpickle import load
from mlutils import eval_class_model
from urllib.request import urlopen

def xgb_test(
    context,
    models_path: DataItem, 
    test_set: DataItem,
    label_column: str,
    plots_dest: str = "plots",
    model_evaluator = None,
    default_model: str = "model.pkl"
) -> None:
    """Test one or more classifier models against held-out dataset
    
    Using held-out test features, evaluates the peformance of the estimated model
    
    Can be part of a kubeflow pipeline as a test step that is run post EDA and 
    training/validation cycles
    
    :param context:         the function context
    :param models_path:     model artifact to be tested
    :param test_set:        test features and labels
    :param label_column:    column name for ground truth labels
    :param plots_dest:      dir for test plots
    :param model_evaluator: WIP: specific method to generate eval, passed in as string
                            or available in this folder
    :param default_model:   'model.pkl', default model artifact file name
    """  
    xtest = test_set.as_df()
    ytest = xtest.pop(label_column)
    
    try:
        model_file, model_obj, _ = get_model(models_path.url, suffix='.pkl')
        model_obj = load(open(model_file, "rb"))
    except ValueError as va: 
        model_obj = load(urlopen(os.path.join(str(models_path), default_model)))
    except Exception as a:
        raise Exception("model location likely specified")

    if not model_evaluator:
        # binary and multiclass
        eval_metrics = eval_class_model(context, xtest, ytest, model_obj)

    # just do this inside log_model?
    model_plots = eval_metrics.pop("plots")
    model_tables = eval_metrics.pop("tables")
    for plot in model_plots:
        context.log_artifact(plot, local_path=f"{plots_dest}/{plot.key}.html")
    for tbl in model_tables:
        context.log_artifact(tbl, local_path=f"{plots_dest}/{plot.key}.csv")

    context.log_results(eval_metrics)

In [ ]:
# nuclio: end-code

### mlconfig

In [ ]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [ ]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("xgb_test")

# add metadata (for templates and reuse)
fn.spec.default_handler = "xgb_test"
fn.spec.description = "test a classifier using held-out or new data"
fn.metadata.categories = ["ml", "test"]
fn.metadata.labels = {"author": "yjb", "framework": "xgboost"}
fn.export("function.yaml")

## tests

In [ ]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/jovyan/data'))

In [ ]:
task_params = {
    "name" : "tasks xgb test",
    "params": {
        "label_column"  : "labels",
        "plots_dest"    : "plots/xgb_test"}}

In [ ]:
TEST_REPO = "https://raw.githubusercontent.com/yjb-ds/testdata/master"
DATA_PATH  = "data/classifier-data.csv"
MODELS_PATH = "models/xgb_test"

### run locally

In [ ]:
from mlrun import run_local, NewTask

run = run_local(NewTask(**task_params),
                handler=xgb_test,
                inputs={"test_set"      : f"{TEST_REPO}/{DATA_PATH}",
                        "models_path"   : f"{TEST_REPO}/{MODELS_PATH}"},
                workdir=mlconf.artifact_path)

In [ ]:
fn.deploy(skip_deployed=True, with_mlrun=False)

run = fn.run(
    NewTask(**task_params),
    inputs={"test_set"      : f"{TEST_REPO}/{DATA_PATH}",
            "models_path"   : f"{TEST_REPO}/{MODELS_PATH}"
        },
    workdir=mlconf.artifact_path)